In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import pathlib
import PIL
from PIL import Image 
from PIL.ImageDraw import Draw

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Processing Data

In [2]:
collected_img = r'.\collectedimages' 
classes = os.listdir(collected_img)
num_classes = len(classes)

print(f'Classes : {classes}\nNum class : {num_classes}')

Classes : ['botolkaca', 'botolplastik', 'kaleng', 'kardus', 'karet', 'kertas', 'plastik', 'sedotan']
Num class : 8


In [108]:
datagen = ImageDataGenerator(
                rotation_range=10,
                width_shift_range=0.1,
                height_shift_range=0.1,
                rescale=1/255.,
                # shear_range=0.2,
                # zoom_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest',
                validation_split=0.15
                )

batch_size = 64

train_gene = datagen.flow_from_directory(
        collected_img,  
        target_size=(224, 224),  
        batch_size=batch_size,
        subset='training',
        classes=classes)

valid_gene = datagen.flow_from_directory(
        collected_img,
        target_size=(224, 224), 
        batch_size=batch_size, 
        subset='validation',
        classes=classes)

Found 4147 images belonging to 8 classes.
Found 729 images belonging to 8 classes.


## Model Creation

In [109]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model

In [110]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# for layer in base_model.layers:
#     layer.trainable = False
trainable_break = 23
for layer in base_model.layers[:trainable_break]:
    # print(layer.name)
    layer.trainable=False
for layer in base_model.layers[trainable_break:]:
    layer.trainable=True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512,activation='relu')(x) 
x = Dropout(.3)(x) 
x = Dense(512,activation='relu')(x) 
x = Dropout(.3)(x) 
preds = Dense(num_classes, activation='sigmoid')(x) 

model=Model(inputs=base_model.input, outputs=preds)

In [111]:
model.summary()

Model: "model_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_36 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                   

In [112]:
# from tensorflow.keras import backend as K

# def Recall(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     recall = true_positives / (possible_positives + K.epsilon())
#     return recall

# def Precision(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     return precision

# def F1Score(y_true, y_pred):
#     precision = Precision(y_true, y_pred)
#     recall = Recall(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [113]:
model.compile(optimizer='adam', 
                loss='categorical_crossentropy', 
                metrics=['accuracy'])

In [114]:
history = model.fit(train_gene, 
                    epochs = 20, 
                    steps_per_epoch = train_gene.samples // batch_size,
                    validation_data = valid_gene, 
                    validation_steps = valid_gene.samples // batch_size)

Epoch 1/20
64/64 [==============================] - 49s 723ms/step - loss: 0.6505 - accuracy: 0.7855 - val_loss: 3.0260 - val_accuracy: 0.3963
Epoch 2/20
64/64 [==============================] - 52s 811ms/step - loss: 0.2243 - accuracy: 0.9300 - val_loss: 4.6573 - val_accuracy: 0.3920
Epoch 3/20
13/64 [=====>........................] - ETA: 36s - loss: 0.2086 - accuracy: 0.9351